data source: https://www.rankedchoicevoting.org/data_clearinghouse

RCV definition: https://ballotpedia.org/Ranked-choice_voting_(RCV)

Additional analysis: 
* http://archive3.fairvote.org/press/san-leandro-facts/
* https://laurendo.wordpress.com/2010/11/24/running-the-numbers/
* http://www.acgov.org/rov/rcv/results/index.htm

Objective for this notebook: separate the elections into the following categories:

1. Leading candidate in the first round has greater than 50% first choice votes
1. Leading candidate in the first round has between 45-50% first choice votes
1. Leading candidate in the first round has less than 45% of first choice votes 

In [1]:
import glob
import pandas
print('pandas',pandas.__version__)

pandas 0.23.4


The Google drive contains folders and subfolders, with .txt files being the desired content 
# First attempt to get data: download individual txt file from Google drive

This approach works, but does not scale well. The manual labor of getting the relevant links would be tedious. Also the filename is not preserved

In [2]:
# https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
# https://github.com/ndrplz/google-drive-downloader
!pip install googledrivedownloader

In [3]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [4]:
gdd.download_file_from_google_drive(file_id='16lMt83ZaI_iLnyr1_R2fx43BimLwi01B',
                                    dest_path='./file.txt')

With a bit of manual intervention, we can get a zip:

# data gathering attempt 2: download all folders from drive manually

Rather than download one file, get a zip file

In a web browser, navigate to all the data: 
https://drive.google.com/drive/folders/1DJzIrTaDW3GSGJTkPTGAlpAMbozFG_pm

Then download all content as a zip. Size is 1.5 GB. Of this, Sante Fe is 1.4GB

I started with just "Alameda County, CA (Berkeley, Oakland, San Leandro)" which is 18MB as a .zip

https://drive.google.com/drive/folders/1u_airJzoLC2PMYMHcF2KYJEKxxKBi5H7

 # get lists of all relevant files
 
 The "voting_data" directory contains the extracted contents of each .zip file

In [82]:
list_of_ballot_files = glob.glob('voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2010/ballot_image_*')
print('number of ballot image files:',len(list_of_ballot_files))

number of ballot image files: 17


we can see what the election is since the filename is descriptive

In [49]:
list_of_ballot_files[0].split('/')[-1].replace('ballot_image_','').strip().replace('.txt','')

'Member, City Council, District 4 - Oakland_Nov 2010'

Later on we will need to determine the names of the candidates. To enable that, load the master_lookup_ files

In [83]:
list_of_lookup_files = glob.glob('voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2010/master_lookup_*')
print('number of master_lookup_ files:',len(list_of_lookup_files))

number of master_lookup_ files: 17


glob's file order doesn't yield paired files

In [48]:
list_of_lookup_files[0].split('/')[-1].replace('master_lookup_','').strip().replace('.txt','')

'Member, City Council, District 1 - Berkeley_Nov 2010'

# create tuple of lookup files and ballot files

In [57]:
list_of_ballots_and_lookups=[]
for this_ballot in list_of_ballot_files:
    ballot_name = this_ballot.split('/')[-1].replace('ballot_image_','').strip().replace('.txt','')
    for this_lookup in list_of_lookup_files:
        lookup_name = this_lookup.split('/')[-1].replace('master_lookup_','').strip().replace('.txt','')
        if ballot_name == lookup_name:
            list_of_ballots_and_lookups.append((this_ballot,this_lookup))

# import candidate data from master_lookup_ files

In [60]:
df = pandas.read_fwf(list_of_ballots_and_lookups[0][1],
                     header=None,
                     widths=[10,7,50,7,7,1,1])
df.columns=['record_id','id','description','list_order','candidates_contest_id','is_writein','is_provisional']
df.shape

(1066, 7)

In [61]:
df.head(10)

,record_id,id,description,list_order,candidates_contest_id,is_writein,is_provisional
0,Candidate,405,JILL BROADHURST,1,37,0,0
1,Candidate,406,RALPH KANZ,2,37,0,0
2,Candidate,407,MELANIE SHELBY,3,37,0,0
3,Candidate,408,LIBBY SCHAAF,4,37,0,0
4,Candidate,409,CLINTON KILLIAN,5,37,0,0
5,Candidate,410,DANIEL SWAFFORD,6,37,0,0
6,Candidate,411,JASON GILLEN,7,37,0,0
7,Candidate,76,Write-In,8,37,1,0
8,Contest,37,"Member, City Council, District 4 - Oakland (RCV)",86,0,0,0
9,Precinct,2,200100,1,0,0,0


In [62]:
df[df['record_id']=='Candidate']

,record_id,id,description,list_order,candidates_contest_id,is_writein,is_provisional
0,Candidate,405,JILL BROADHURST,1,37,0,0
1,Candidate,406,RALPH KANZ,2,37,0,0
2,Candidate,407,MELANIE SHELBY,3,37,0,0
3,Candidate,408,LIBBY SCHAAF,4,37,0,0
4,Candidate,409,CLINTON KILLIAN,5,37,0,0
5,Candidate,410,DANIEL SWAFFORD,6,37,0,0
6,Candidate,411,JASON GILLEN,7,37,0,0
7,Candidate,76,Write-In,8,37,1,0


# import ballot data

In [63]:
df = pandas.read_fwf(list_of_ballots_and_lookups[0][0],
                     header=None,
                     widths=[7,9,7,3,7,3,7,1,1])
df.columns=['contest_id','pref_voter_id',
            'serial_number','tally_type_id',
            'precinct_id','vote_rank',
            'candidate_id','over_vote','under_vote']
df.shape

(71652, 9)

In [64]:
df.head()

,contest_id,pref_voter_id,serial_number,tally_type_id,precinct_id,vote_rank,candidate_id,over_vote,under_vote
0,37,12307,2,3,152,1,408,0,0
1,37,12307,2,3,152,2,409,0,0
2,37,12307,2,3,152,3,406,0,0
3,37,12313,6,3,293,1,411,0,0
4,37,12313,6,3,293,2,408,0,0


# analyze ballot dataframe

In [10]:
df.nunique()

contest_id           1
pref_voter_id    23884
serial_number       81
tally_type_id        5
precinct_id         45
vote_rank            3
candidate_id         9
over_vote            2
under_vote           2
dtype: int64

* There are 81 unique voting machines
* There are 23,885 voters
* There are 9 candidates

In [11]:
# what are the candidate IDs?
df['candidate_id'].unique()

array([408, 409, 406, 411, 410,   0, 405, 407,  76])

In [12]:
# how many rows does each candidate appear in?
df['candidate_id'].value_counts()

0      18090
408    14790
405    12113
410     7521
407     7468
409     5019
406     3529
411     2723
76       399
Name: candidate_id, dtype: int64

In [13]:
df[df['candidate_id']==0].head()

,contest_id,pref_voter_id,serial_number,tally_type_id,precinct_id,vote_rank,candidate_id,over_vote,under_vote
8,37,12769,7,3,152,3,0,0,1
17,37,12772,7,3,152,3,0,0,1
18,37,12773,7,3,152,1,0,0,1
19,37,12773,7,3,152,2,0,0,1
20,37,12773,7,3,152,3,0,0,1


In [14]:
# drop rows where candidate_id==0

df_cand = df[df['candidate_id']!=0]

### did any candidate win the first round?

In [15]:
# compare only the candidates with rank==1
df_cand[df_cand['vote_rank']==1].head()

,contest_id,pref_voter_id,serial_number,tally_type_id,precinct_id,vote_rank,candidate_id,over_vote,under_vote
0,37,12307,2,3,152,1,408,0,0
3,37,12313,6,3,293,1,411,0,0
6,37,12769,7,3,152,1,408,0,0
9,37,12770,7,3,152,1,406,0,0
12,37,12771,7,3,152,1,408,0,0


In [16]:
# https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
# https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/
df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()

candidate_id
76       67
405    4794
406     878
407    2448
408    8746
409    1133
410    2345
411     526
Name: vote_rank, dtype: int64

In RCV, if a candidate wins a majority of first-preference votes, that candidate is declared the winner.

Caveat: this dataframe ignores rows where no candidate preference was provided

In [17]:
# majority = half the sum of first choices 
majority_first_round = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count().sum()/2
majority_first_round

10468.5

In [18]:
df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()>majority_first_round

candidate_id
76     False
405    False
406    False
407    False
408    False
409    False
410    False
411    False
Name: vote_rank, dtype: bool

Since no candidate won the majority of the first round, the next step in the algorithm is to eliminate the candidate with the lowest count.

In [19]:
df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count().idxmin()

76

In [20]:
df_second_round = df_cand[df_cand['candidate_id']!=76]
df_second_round.head()

,contest_id,pref_voter_id,serial_number,tally_type_id,precinct_id,vote_rank,candidate_id,over_vote,under_vote
0,37,12307,2,3,152,1,408,0,0
1,37,12307,2,3,152,2,409,0,0
2,37,12307,2,3,152,3,406,0,0
3,37,12313,6,3,293,1,411,0,0
4,37,12313,6,3,293,2,408,0,0


Then first-preference votes cast for the failed candidate are eliminated, lifting the second-preference choices indicated on those ballots. 

However, for my analysis I only care about the first round. Recall my objective is to categorize elections by
* Leading candidate in the first round has greater than 50% first choice votes
* Leading candidate in the first round has between 45-50% first choice votes
* Leading candidate in the first round has less than 45% of first choice votes

In [21]:
number_of_first_choice_votes = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count().sum()

In [22]:
outcome = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()>number_of_first_choice_votes*0.5
outcome

candidate_id
76     False
405    False
406    False
407    False
408    False
409    False
410    False
411    False
Name: vote_rank, dtype: bool

In [23]:
outcome.any()

False

# evaluate condorcet criterion

task: Merge the ballot data with the candidate names to see if a pair-wise winner actually won an election

"The Condorcet criterion is satisfied if when a candidate can beat every other candidate in a head-to-head election, that beat-all candidate always wins. "

--> To test this condition in a race with 5 candidates, compare each pair and use first ranked choices only.

In [86]:
# ballot data
df = pandas.read_fwf(list_of_ballots_and_lookups[0][0],
                     header=None,
                     widths=[7,9,7,3,7,3,7,1,1])
df.columns=['contest_id','pref_voter_id',
            'serial_number','tally_type_id',
            'precinct_id','vote_rank',
            'candidate_id','over_vote','under_vote']
df.shape

df_cand = df[df['candidate_id']!=0]

In [87]:
# count votes from first round
candidates_id_and_votes = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()
candidates_id_and_votes

candidate_id
76       67
405    4794
406     878
407    2448
408    8746
409    1133
410    2345
411     526
Name: vote_rank, dtype: int64

In [88]:
# to get names, load from master_lookup_

df = pandas.read_fwf(list_of_ballots_and_lookups[0][1],
                     header=None,
                     widths=[10,7,50,7,7,1,1])
df.columns=['record_id','id','description','list_order','candidates_contest_id','is_writein','is_provisional']
candidate_lookup_df = df[df['record_id']=='Candidate']
candidate_lookup_df

,record_id,id,description,list_order,candidates_contest_id,is_writein,is_provisional
0,Candidate,405,JILL BROADHURST,1,37,0,0
1,Candidate,406,RALPH KANZ,2,37,0,0
2,Candidate,407,MELANIE SHELBY,3,37,0,0
3,Candidate,408,LIBBY SCHAAF,4,37,0,0
4,Candidate,409,CLINTON KILLIAN,5,37,0,0
5,Candidate,410,DANIEL SWAFFORD,6,37,0,0
6,Candidate,411,JASON GILLEN,7,37,0,0
7,Candidate,76,Write-In,8,37,1,0


In [89]:
# now we can merge names with first round votes
candidates_and_votes={}
for cand_id, vote_count in candidates_id_and_votes.items():
    candidates_and_votes[candidate_lookup_df[candidate_lookup_df['id']==cand_id]['description'].values[0]]=vote_count

candidates_and_votes

{'Write-In': 67,
 'JILL BROADHURST': 4794,
 'RALPH KANZ': 878,
 'MELANIE SHELBY': 2448,
 'LIBBY SCHAAF': 8746,
 'CLINTON KILLIAN': 1133,
 'DANIEL SWAFFORD': 2345,
 'JASON GILLEN': 526}

In [90]:
# run the pair wise comparison
for candidate_a, votes_for_a in candidates_and_votes.items():
    print(candidate_a)
    candidate_a_won_all=True
    for candidate_b, votes_for_b in candidates_and_votes.items():
        if candidate_a!=candidate_b:
            if votes_for_a>votes_for_b:
                print('candidate '+str(candidate_a)+' wins against '+str(candidate_b))
            else:
                print('candidate '+str(candidate_a)+' loses against '+str(candidate_b))
                candidate_a_won_all=False
    if candidate_a_won_all:
        print('    WINNER = ',candidate_a)

Write-In
candidate Write-In loses against JILL BROADHURST
candidate Write-In loses against RALPH KANZ
candidate Write-In loses against MELANIE SHELBY
candidate Write-In loses against LIBBY SCHAAF
candidate Write-In loses against CLINTON KILLIAN
candidate Write-In loses against DANIEL SWAFFORD
candidate Write-In loses against JASON GILLEN
JILL BROADHURST
candidate JILL BROADHURST wins against Write-In
candidate JILL BROADHURST wins against RALPH KANZ
candidate JILL BROADHURST wins against MELANIE SHELBY
candidate JILL BROADHURST loses against LIBBY SCHAAF
candidate JILL BROADHURST wins against CLINTON KILLIAN
candidate JILL BROADHURST wins against DANIEL SWAFFORD
candidate JILL BROADHURST wins against JASON GILLEN
RALPH KANZ
candidate RALPH KANZ wins against Write-In
candidate RALPH KANZ loses against JILL BROADHURST
candidate RALPH KANZ loses against MELANIE SHELBY
candidate RALPH KANZ loses against LIBBY SCHAAF
candidate RALPH KANZ loses against CLINTON KILLIAN
candidate RALPH KANZ lo

In [79]:
# which race was this?
list_of_ballots_and_lookups[0]

('voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2010/ballot_image_ Member, City Council, District 4 - Oakland_Nov 2010.txt',
 'voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2010/master_lookup_ Member, City Council, District 4 - Oakland_Nov 2010.txt')

According to the site http://www.smartvoter.org/2010/11/02/ca/alm/race/018/<BR>
    and https://www.eastbayexpress.com/SevenDays/archives/2010/11/05/november-2010-election-results<BR>
LIBBY SCHAAF won the election

# one year has a different format

In [109]:
def vote_table(ballot_file):
    fwf=False
    with open(ballot_file,'r') as fil:
        file_contents = fil.readlines()
        if len(file_contents[0].strip())==45:
            fwf=True
    
    if fwf:
        df = pandas.read_fwf(ballot_file,
                         header=None,
                         widths=[7,9,7,3,7,3,7,1,1])
        df.columns=['contest_id','pref_voter_id',
            'serial_number','tally_type_id',
            'precinct_id','Vote_Rank',
            'CandidateID','over_vote','under_vote']
        df_cand = df[df['CandidateID']!=0] # drop rows where no candidate is specified
    else:
        df_cand = pandas.read_csv(ballot_file, sep='\t', engine='python')#,delim_whitespace=True)

    return df_cand

In [112]:
df = vote_table('voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2010/ballot_image_Mayor of Oakland_Nov 2010.txt')

In [113]:
df.head()

,contest_id,pref_voter_id,serial_number,tally_type_id,precinct_id,vote_rank,candidate_id,over_vote,under_vote
0,29,12307,2,3,152,1,399,0,0
1,29,12307,2,3,152,2,402,0,0
2,29,12307,2,3,152,3,397,0,0
3,29,12308,2,3,250,1,393,0,0
4,29,12308,2,3,250,2,397,0,0


In [110]:
df = vote_table('voting_data/Alameda/Alameda (Oakland, San Leandro, Berkeley) 2014/ballot_image_Member, City Council, District 2 - Oakland_Nov 2014.txt')

In [111]:
df

,Contest_ID,Pref_Voter_ID,Serial Number,Tally_Type,Precinct_ID,Vote_Rank,CandidateID,Over-Vote,Under_Vote,Vote Rank2,CandID2,OverVote,UnderVote,VoteRank3,CandID3,OverVote.1,UnderVote.1
0,85,77123,143,1,143,1,437,0,0,2,435,0,0,3,434,0,0
1,85,77124,143,1,143,1,0,0,1,2,0,0,1,3,0,0,1
2,85,77125,143,1,143,1,433,0,0,2,435,0,0,3,437,0,0
3,85,77126,143,1,143,1,433,0,0,2,437,0,0,3,435,0,0
4,85,77127,143,1,143,1,0,0,1,2,0,0,1,3,0,0,1
5,85,77128,143,1,143,1,433,0,0,2,0,0,1,3,0,0,1
6,85,77129,143,1,143,1,433,0,0,2,0,0,1,3,0,0,1
7,85,77130,143,1,143,1,433,0,0,2,0,0,1,3,0,0,1
8,85,77131,143,1,143,1,433,0,0,2,437,0,0,3,0,0,1
9,85,77132,143,1,143,1,0,0,1,2,0,0,1,3,0,0,1
